In [1]:
%%writefile Dockerfile
FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-inference:2.1.0-transformers4.37.0-gpu-py310-cu118-ubuntu20.04
RUN pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
RUN pip install flash-attn --no-build-isolation
RUN pip install transformers==4.46.3 accelerate==1.0.1
RUN pip install decord ffmpeg-python imageio opencv-python
RUN apt-get update && apt-get install libgl1
RUN apt update && apt install ffmpeg

Writing Dockerfile


In [10]:
%%sh

# Specify a name and a tag
algorithm_name=huggingface-pytorch-inference
tag=2.1.0-transformers4.37.0-gpu-py310-cu118-ubuntu20.04

account=$(aws sts get-caller-identity --query Account --output text)


# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=us-west-2

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${tag}"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Log into Docker
sudo aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build -t ${algorithm_name} --network sagemaker .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

sudo: docker: command not found

Unable to locate credentials. You can configure credentials by running "aws configure".
DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            BuildKit is currently disabled; enable it by removing the DOCKER_BUILDKIT=0
            environment-variable.



Sending build context to Docker daemon  20.99kB
Step 1/8 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-inference:2.1.0-transformers4.37.0-gpu-py310-cu118-ubuntu20.04


Head "https://763104351884.dkr.ecr.us-west-2.amazonaws.com/v2/huggingface-pytorch-inference/manifests/2.1.0-transformers4.37.0-gpu-py310-cu118-ubuntu20.04": no basic auth credentials
Error response from daemon: No such image: huggingface-pytorch-inference:latest
